In [2]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
import re

In [3]:
analyser = SentimentIntensityAnalyzer()

def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    print("{:-<40} {}".format(sentence, str(score)))

In [4]:
length = "short"
file = pd.read_csv("matches3.csv", sep=',', header=None, names=['word', 'text'])
dataset = file.drop_duplicates().reset_index(drop=True)

In [5]:
found_sents = []
joint_word = []

for index, row in dataset.iterrows():
        sentences_list = row.text.split('.')
        for sentence in sentences_list:
            # Check if there is an exact match of the word in the sentence
            if re.search(r'\b' + row['word'] + r'\b', sentence):
                found_sents.append(sentence)
                joint_word.append(row['word'])

In [9]:
sentiment_df = pd.DataFrame(columns=['word', 'sent'])
sentiment_df['word'] = joint_word
sentiment_df['sent'] = found_sents

In [10]:
scores = []
for index, row in sentiment_df.iterrows():
    scores.append(analyser.polarity_scores(row.sent))
    
sentiment_df['neg'] = [item['neg'] for item in scores]
sentiment_df['neu'] = [item['neu'] for item in scores]
sentiment_df['pos'] = [item['pos'] for item in scores]
sentiment_df['comp'] = [item['compound'] for item in scores]

In [16]:
sentiment_df.groupby(['word'])[['neg', 'neu', 'pos', 'comp']].mean().sort_values(by=['neg', 'pos'], ascending=[False, True])

,neg,neu,pos,comp
word,,,,
gag whore,0.770,0.230000,0.000000,-0.771700
statto,0.608,0.392000,0.000000,-0.476700
thieving bastard,0.412,0.588000,0.000000,-0.542300
fallen champion,0.391,0.000000,0.609000,0.340000
stuck key,0.384,0.539000,0.077000,-0.074500
...,...,...,...,...
stylista,0.000,0.858889,0.141111,0.220867
berliner ballen,0.000,0.795000,0.205000,0.850400
hunkie,0.000,0.650000,0.350000,0.790100


In [33]:
def print_sent_by_word(word):
    print(sentiment_df[sentiment_df['word'] == word][['word', 'sent']])
    print('')

In [34]:
def print_text_by_word(word):
    print(dataset[dataset['word'] == word]['text'].values)
    print('')

In [35]:
search_word = 'gag whore'
print_sent_by_word(search_word)
print_text_by_word(search_word)

          word                      sent
288  gag whore  deepthroat gag whore cum

["princess cierra femdom - adult videos.home.categories.pantyhose.blowbang.bath.pool.massage.femdom.high heels.centerfold.cum in pussy.mom.panties.indian.blindfold.boots.fetish.shorts.beach.spreading.schoolgirl.nurse.cum in mouth.yoga pants.european.cosplay.titjob.double penetration.oiled.face.footjob.blowjob.dildo.facesitting.creampie.bbw.bikini.skirt.humping.pornstar.wife.seduction.all pics.sex dating.princess cierra femdom.saturday 10th march she is a pro when it comes to role-playing. browse princess sierra's phone sex listings and goody offerings  found in the financial domination category. she can be playful but do not be fooled by her innocent face because she can tease your cock till you cum in her face. you can be the high school virgin you have always desired  or you can be that dominant boss in a femdom who gets his slaves caged and whoops their ass into a frenzy..marsha. age: 21. hey guys its